In [3]:
import numpy as np
import pandas as pd

import rasterio as rio
import tensorflow as tf
import matplotlib.pyplot as plt

import joblib
import pickle
import pyproj

print(tf.__version__)

2.16.1


In [ ]:
model = tf.keras.models.load_model('map_model\Flood_map_model.keras')


In [ ]:
test_upstream = [0.0, 0.2]
test_downstream = [0.0,0.3]
test_upstream_interpolate = np.linspace(test_upstream[0],test_upstream[1], num=9)
test_downstream_interpolate = np.linspace(test_downstream[0],test_downstream[1], num=9)
stream_data = [test_upstream_interpolate[0],test_downstream_interpolate[0]]+list(test_upstream_interpolate[1:])+list(test_downstream_interpolate[1:])
stream_df =  pd.DataFrame([stream_data])

In [ ]:
with open('model/scaler.pkl', 'rb') as f:
    scaler = pickle.load(f)

with open('model/profile_data.pkl', 'rb') as f:
    profile = pickle.load(f)

In [ ]:
test1 = stream_df
test1_scaled = scaler.transform(test1)


stream_scaled = scaler.transform(stream_df)
stream_scaled = stream_scaled.reshape(1, 1, 18)
pred = model.predict(stream_scaled)
pred.resize(600, 900)
pred[pred < 0.15] = 0
with rio.Env():
    profile.update(dtype=str(pred.dtype), count=1, compress='lzw')
    with rio.open("test_map.asc", 'w', **profile) as dst:
        dst.write(pred, 1)

In [ ]:
with rio.open("test_map.asc") as real_data:  # Use a context manager to close the file
    real_band = real_data.read(1)
    plt.imshow(real_band)
    plt.colorbar()        
    plt.show()